安装软件包

In [63]:
%pip install -U tensorflow pillow "jax[cuda12]" flax

/home/pdch/miniforge3/envs/jax/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.


获取NEU Seg数据集

In [3]:
!git clone https://github.com/DHW-Master/NEU_Seg.git data && rm -rf data/.git

正克隆到 'data'...
remote: Enumerating objects: 8947, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 8947 (delta 0), reused 0 (delta 0), pack-reused 8945 (from 1)
接收对象中: 100% (8947/8947), 35.08 MiB | 5.38 MiB/s, 完成.


对数据集进行处理

In [11]:
import os
import numpy as np
from PIL import Image

# 文件夹路径
train_image_dir = 'data/images/training'
train_mask_dir = 'data/annotations/training'
test_image_dir = 'data/images/test'
test_mask_dir = 'data/annotations/test'

# 获取文件列表
train_images = sorted(os.listdir(train_image_dir))
train_masks = sorted(os.listdir(train_mask_dir))
test_images = sorted(os.listdir(test_image_dir))
test_masks = sorted(os.listdir(test_mask_dir))

# 确保文件对应
assert len(train_images) == len(train_masks), "训练图片和掩码数量不一致"
assert len(test_images) == len(test_masks), "测试图片和掩码数量不一致"

# 完整路径
train_image_paths = [os.path.join(train_image_dir, img) for img in train_images]
train_mask_paths = [os.path.join(train_mask_dir, msk) for msk in train_masks]
test_image_paths = [os.path.join(test_image_dir, img) for img in test_images]
test_mask_paths = [os.path.join(test_mask_dir, msk) for msk in test_masks]

def load_images(paths):
    data = []
    for path in paths:
        img = Image.open(path).convert('RGB')
        img_array = np.array(img)
        # img_array = np.transpose(img_array, (2, 0, 1))  # 将通道轴移到最前面
        data.append(img_array)
    return np.array(data)

def load_masks(paths):
    data = []
    for path in paths:
        img = Image.open(path).convert('L')
        data.append(np.array(img))
    return np.array(data)

# 加载并保存训练集
train_images = load_images(train_image_paths)
train_masks = load_masks(train_mask_paths)
np.save('data/train_images.npy', train_images)
np.save('data/train_masks.npy', train_masks)

# 加载并保存测试集
test_images = load_images(test_image_paths)
test_masks = load_masks(test_mask_paths)
np.save('data/test_images.npy', test_images)
np.save('data/test_masks.npy', test_masks)

确认jax状态

In [1]:
import jax

# Show on which platform JAX is running.
print("JAX running on", jax.devices()[0].platform.upper())

JAX running on GPU


# 运行

In [19]:
!python main.py

2025-01-17 11:43:38.634151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737085418.642929   20327 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737085418.645456   20327 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1737085421.465511   20327 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6195 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
Epoch 1/500: 0it [00:00, ?it/s]<class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>
Train loss: Traced<ShapedArray(float32[])>with<DynamicJaxprTrace>
2025-01-17 11:43:46.863415: W 

清理log

In [18]:
!rm -rf work_dir/*